In [1]:
#USING NEURAL NETWORK + RoBERTA

import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm, datasets
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns

from sklearn import metrics

from transformers import RobertaConfig, RobertaModel
from transformers import RobertaTokenizer

from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

In [2]:
df = pd.read_excel(r"Data_600_Labeled_Final.xlsx")
df['innovation_num']=df.YN_INNOVATION.eq('yes').mul(1)

In [3]:
df = df.sort_values(by='YN_INNOVATION', ascending=False)

In [4]:
df = df.sort_values(by='YN_INNOVATION', ascending=False)
df2 = df[223:].sample(n=222)


In [5]:
df.drop(df.tail(378).index, inplace = True)
df = df.append(df2)
df

,Unnamed: 0,an,title,snippet,body,full_text,"Does it mention an innovation (yes,no,unsure)?",What is the company?,"Where is company mentioned (title,body,snippet,all)?",What is the product name?,"Where is the product name mentioned(title,body,snippet,all)","What could indicate that this is a product (for use in our algorithms, e.g. TM,R,C,CAPITAL LETTERS, Quotes,Used with certain key phrases or words etc.)?","Features of the product, if mentioned",assignee,YN_INNOVATION,innovation_num
0,0,PNN0000020180426ee4p00006,Creaform and Metrologic Group Announce Worldwi...,Metrologic Group and Creaform have entered int...,Speed and volumetric accuracy\n\nThe CUBE-R™ e...,Creaform and Metrologic Group Announce Worldwi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,yes,1
542,542,BWR0000020161027ecar0006r,ServiceNow Launches Business Command Center fo...,ITBM Suite combines application rationalizatio...,"ServiceNow (NYSE:NOW), the enterprise cloud co...",ServiceNow Launches Business Command Center fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,yes,1
425,425,FLYWAL0020200715eg7f0187p,17:14 EDT Amazon Web Services announces genera...,17:14 EDT Amazon Web Services announces genera...,There are no additional charges or upfront co...,17:14 EDT Amazon Web Services announces genera...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,yes,1
513,513,COMWKN0020150306eb3c000ji,Yardi; Yardi Unveils New CRM Solution for Seni...,2015 MAR 12 (VerticalNews) -- By a News Report...,Data generated through Yardi Senior CRM flows ...,Yardi; Yardi Unveils New CRM Solution for Seni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,yes,1
379,379,WLNW000020140718ea7i0004z,Layered Tech to Introduce New Federal Communit...,At the Microsoft Worldwide Partner Conference ...,"""Layered Tech's expertise and experience with ...",Layered Tech to Introduce New Federal Communit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK,yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,474,CMCIOL0020140625ea6p00007,Symantec launches initiative to close cybersec...,A pilot of the program will start in August in...,A pilot of the program will start in August in...,Symantec launches initiative to close cybersec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,no,0
375,375,DJDN000020160729ec7t0007i,"Tech Stock Roundup: Amazon, Alphabet and Faceb...",Our U.S. colleague Tiernan Ray had a busy day ...,"""The team in India is inventing at a torrid pa...","Tech Stock Roundup: Amazon, Alphabet and Faceb...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK,no,0
424,424,COMWKN0020170831ed8v000k0,Patents; Researchers Submit Patent Application...,2017 SEP 6 (VerticalNews) -- By a News Reporte...,News editors obtained the following quote from...,Patents; Researchers Submit Patent Application...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,no,0
177,177,INSRWK0020160205ec250001e,CSC; CSC Receives XCelent Award for EMEA Polic...,2016 FEB 12 (VerticalNews) -- By a News Report...,The report uses Celent's proprietary framework...,CSC; CSC Receives XCelent Award for EMEA Polic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no,0


In [6]:
# Initializing a RoBERTa configuration
configuration = RobertaConfig()

# Initializing a model from the configuration
model = RobertaModel(configuration)

# Accessing the model configuration
configuration = model.config

In [7]:
from transformers import RobertaTokenizer
model_class, tokenizer_class, pretrained_weights = (AutoModel, AutoTokenizer,'roberta-base')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
tokenized = df['full_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=512, truncation=True)))

In [9]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [10]:
np.array(padded).shape

(444, 512)

In [11]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(444, 512)

In [12]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [13]:
features = last_hidden_states[0][:,0,:].numpy()
labels = df['innovation_num']

In [14]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [18]:
from sklearn.neural_network import MLPClassifier

nn_clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 5), random_state=1, warm_start=True, max_iter=10000)

In [19]:
nn_clf.fit(train_features, train_labels)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 5), max_iter=10000,
              random_state=1, solver='sgd', warm_start=True)

In [20]:
nn_clf.score(test_features, test_labels)

0.46846846846846846